In [1]:
# Import libraries
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd

In [2]:
# Specify filepaths and filenames.
filepath = '/home/jovyan/shared-public/MLinGlaciology/BYOP/data/point_mb_glaciers_norway/'
filename = 'stake_mb_norway_cleaned_ids.csv'

In [3]:
# Load data.
data = pd.read_csv(filepath + filename, sep=';')

data
data.columns

Index(['RGIID', 'GLIMSID', 'BREID', 'utm_zone', 'utm_east_approx',
       'utm_north_approx', 'altitude_approx', 'location_description',
       'location_id', 'stake_no', 'altitude', 'utm_east', 'utm_north',
       'balance_winter', 'balance_summer', 'balance_netto',
       'dt_prev_year_min_date', 'dt_curr_year_max_date',
       'dt_curr_year_min_date', 'stake_remark', 'flag_correction',
       'approx_loc', 'approx_altitude', 'diff_north', 'diff_east',
       'diff_altitude', 'diff_netto'],
      dtype='object')

In [5]:
# Transform UTM coordinates to lat,long (WGS84)

from pyproj import Transformer

# WGS84 Latitude/Longitude: "EPSG:4326"
# EUREF89 UTM32: EPSG:25832
# EUREF89 UTM33: EPSG:25833
# EUREF89 UTM34: EPSG:25834

#data_coords = data[['utm_zone','utm_east_approx','utm_north_approx','utm_east','utm_north']]
data.loc[:, ('lat')] = np.nan
data.loc[:, ('lon')] = np.nan
data.loc[:, ('lat_approx')] = np.nan
data.loc[:, ('lon_approx')] = np.nan

#inProj = Proj('epsg:3857')
#outProj = Proj('epsg:4326')
#x1,y1 = -11705274.6374,4826473.6922
#x2,y2 = transform(inProj,outProj,x1,y1)
#print x2,y2

# Define output projection.
outProj = "EPSG:4326"

for i in range(data.index.start, data.index.stop):
 
    # Check UTM zone of coordinates and set initial projection.
    if data.iloc[i]['utm_zone'] == 34:
        inProj = "EPSG:25834"
    elif data.iloc[i]['utm_zone'] == 33:
        inProj = "EPSG:25833"
    elif data.iloc[i]['utm_zone'] == 32:
        inProj = "EPSG:25832"
    else: 
        print('UTM zone not recognized.')
    
    # Define transformer.
    transformer = Transformer.from_crs(inProj, outProj, always_xy=True)

    # Get UTM coordinates
    x,y = data.iloc[i]['utm_east'],data.iloc[i]['utm_north']
    x_a,y_a = data.iloc[i]['utm_east_approx'],data.iloc[i]['utm_north_approx']

    # Project to lat,lon (WGS84) coordinates
    x_p,y_p = transformer.transform(x,y)
    x_ap,y_ap = transformer.transform(x_a,y_a)
    
    # Add to dataframe
    data.loc[i, ('lat')] = y_p
    data.loc[i, ('lon')] = x_p
    data.loc[i, ('lat_approx')] = y_ap
    data.loc[i, ('lon_approx')] = x_ap


In [6]:
data

,RGIID,GLIMSID,BREID,utm_zone,utm_east_approx,utm_north_approx,altitude_approx,location_description,location_id,stake_no,...,approx_loc,approx_altitude,diff_north,diff_east,diff_altitude,diff_netto,lat,lon,lat_approx,lon_approx
0,RGI60-08.01258,G021737E70130N,54,34,529400,7780050,500,Nederst pÃ¥ bretunga,L10,L1012,...,N,N,50,80,-29.0,0.000000e+00,70.125889,21.777016,70.125449,21.774891
1,RGI60-08.01258,G021737E70130N,54,34,529400,7780050,500,Nederst pÃ¥ bretunga,L10,L1013,...,N,N,38,53,-61.0,0.000000e+00,70.125784,21.776300,70.125449,21.774891
2,RGI60-08.01258,G021737E70130N,54,34,529400,7780050,500,Nederst pÃ¥ bretunga,L10,L1013,...,N,N,32,38,-56.0,0.000000e+00,70.125732,21.775903,70.125449,21.774891
3,RGI60-08.01258,G021737E70130N,54,34,529400,7780050,500,Nederst pÃ¥ bretunga,L10,L1013,...,N,N,32,38,-56.0,0.000000e+00,70.125732,21.775903,70.125449,21.774891
4,RGI60-08.01258,G021737E70130N,54,34,529400,7780050,500,Nederst pÃ¥ bretunga,L10,L1016,...,N,N,9,-52,-32.0,0.000000e+00,70.125536,21.773523,70.125449,21.774891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,RGI60-08.02969,G006362E59989N,3141,32,351800,6647500,1080,Nederst pÃ¥ bretunga,B15,B1509,...,N,N,-20,-10,-2.0,0.000000e+00,59.938030,6.347331,59.938213,6.347495
4042,RGI60-08.02969,G006362E59989N,3141,32,351800,6647500,1080,Nederst pÃ¥ bretunga,B15,B1509,...,N,N,-35,-13,-2.0,4.440000e-16,59.937895,6.347288,59.938213,6.347495
4043,RGI60-08.02969,G006362E59989N,3141,32,351800,6647500,1080,Nederst pÃ¥ bretunga,B15,B1514,...,N,N,-6,-27,-1.0,0.000000e+00,59.938150,6.347017,59.938213,6.347495
4044,RGI60-08.02969,G006362E59989N,3141,32,351800,6647500,1080,Nederst pÃ¥ bretunga,B15,B1514,...,N,N,-20,-15,-2.0,0.000000e+00,59.938029,6.347241,59.938213,6.347495


In [7]:
data.to_csv(filepath + 'stake_mb_norway_cleaned_ids_latlon.csv', index=False)  


In [10]:
# Open dataframe with observations and coordinates

# Specify filepaths and filenames.
filepath = '/home/jovyan/shared-public/MLinGlaciology/BYOP/data/point_mb_glaciers_norway/'
filename = 'stake_mb_norway_cleaned_ids_latlon.csv'

# Load data
point_data = pd.read_csv(filepath + filename)#, sep=';')
point_data

,RGIID,GLIMSID,BREID,utm_zone,utm_east_approx,utm_north_approx,altitude_approx,location_description,location_id,stake_no,...,approx_loc,approx_altitude,diff_north,diff_east,diff_altitude,diff_netto,lat,lon,lat_approx,lon_approx
0,RGI60-08.01258,G021737E70130N,54,34,529400,7780050,500,Nederst pÃ¥ bretunga,L10,L1012,...,N,N,50,80,-29.0,0.000000e+00,70.125889,21.777016,70.125449,21.774891
1,RGI60-08.01258,G021737E70130N,54,34,529400,7780050,500,Nederst pÃ¥ bretunga,L10,L1013,...,N,N,38,53,-61.0,0.000000e+00,70.125784,21.776300,70.125449,21.774891
2,RGI60-08.01258,G021737E70130N,54,34,529400,7780050,500,Nederst pÃ¥ bretunga,L10,L1013,...,N,N,32,38,-56.0,0.000000e+00,70.125732,21.775903,70.125449,21.774891
3,RGI60-08.01258,G021737E70130N,54,34,529400,7780050,500,Nederst pÃ¥ bretunga,L10,L1013,...,N,N,32,38,-56.0,0.000000e+00,70.125732,21.775903,70.125449,21.774891
4,RGI60-08.01258,G021737E70130N,54,34,529400,7780050,500,Nederst pÃ¥ bretunga,L10,L1016,...,N,N,9,-52,-32.0,0.000000e+00,70.125536,21.773523,70.125449,21.774891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,RGI60-08.02969,G006362E59989N,3141,32,351800,6647500,1080,Nederst pÃ¥ bretunga,B15,B1509,...,N,N,-20,-10,-2.0,0.000000e+00,59.938030,6.347331,59.938213,6.347495
4042,RGI60-08.02969,G006362E59989N,3141,32,351800,6647500,1080,Nederst pÃ¥ bretunga,B15,B1509,...,N,N,-35,-13,-2.0,4.440000e-16,59.937895,6.347288,59.938213,6.347495
4043,RGI60-08.02969,G006362E59989N,3141,32,351800,6647500,1080,Nederst pÃ¥ bretunga,B15,B1514,...,N,N,-6,-27,-1.0,0.000000e+00,59.938150,6.347017,59.938213,6.347495
4044,RGI60-08.02969,G006362E59989N,3141,32,351800,6647500,1080,Nederst pÃ¥ bretunga,B15,B1514,...,N,N,-20,-15,-2.0,0.000000e+00,59.938029,6.347241,59.938213,6.347495
